## <b> Modelling HDB Resale Prices </b>
---
### <b> Notebook 2: Data Processing </b>
---

<b> Project Overview / Objectives </b>

* Build a model that predicts the resale price of public housing (i.e., Housing Development Board (HDB)) apartments in Singapore
    * Evaluate the model's predictive ability against unseen "test" data, via a Kaggle submission
* Separably identify key features associated with resale prices - by quantifying their importance from a statistical and practical viewpoint

<br>
<b> Notebook Overview </b>

* The codes in this notebook perform the following data processing functions:
    * <b>'Simulated Test' dataset creation</b> 
    * <b>Data cleaning</b>: to exclude outliers, manage missing values
    * <b>Feature engineering</b>: to construct additional features hypothesized to be related to resale prices
    * <b>Feature retention</b>: to focus on features hypothesized to be related to resale prices
    * <b>Data scaling</b>: scaling of numeric features required for implementing regression regularisation techniques

<br>
<b> Notebook Structure </b>

* Part 1: Import Data, Create 'Simulated Test' Dataset
* Part 2: Define Functions: Data Cleaning 
* Part 3: Define Functions: Feature Engineering
* Part 4: Define Functions: Others
* Part 5: Perform Data Processing, Export Processed Data as CSV Files

<br>


---

<b>Imports & Installations</b>

In [1]:
# Import Libraries
import numpy as np
import pandas as pd

import matplotlib.pyplot as plt
import seaborn as sns

from sklearn.model_selection import train_test_split
from sklearn.preprocessing import OneHotEncoder, StandardScaler



---

#### <b> Part 1: Import Data, Create Simulated Test Dataset </b>

<b>Data Overview (Recap)</b>

Data on HDB resale prices over ~ 9 years (2012-03 to 2021-04), split into:

* <b>Training Dataset</b> [train.csv]: ~ 150,000 observations (resale price data: present)
* <b>Testing Dataset</b> [test.csv]: ~ 16,500 observations   (resale price data: masked)
    * *Resale price data has been intentionally masked*
    * *Procedurally, a trained model is meant to be applied on the "kaggle testing" dataset, and its performance evaluated via a Kaggle submission*

<br>
<b>Background: Creating a Simulated Test Dataset</b>

* A 'Simulated Test' dataset will be subtracted from the "training dataset [train.csv]" to aid with model evaluation prior to its application to the "testing dataset [test.csv]" (for Kaggle submission)
* Creation of the 'Simulated Test' dataset will precede the formulation of data cleaning procedures, to prevent data leakage across 'Training'-'Simulated Test'-'Testing' datasets

<br>


---
<b>(a) Import Data</b>

In [2]:
# Import Raw Data
df_train_init = pd.read_csv("1_Data/train.csv", dtype={41:str})
df_test = pd.read_csv("1_Data/test.csv", dtype={40:str})

<b>(b) Create 'Simulated Test' Dataset</b>

In [3]:
# Subtract 'Simulated Test' Dataset from "Training" Dataset

# 1. Specify X and y Features
features = [feature for feature in df_train_init.columns]
features.remove("resale_price")

X = df_train_init[features]
y = df_train_init["resale_price"]

# 2. Subtract 'Simulated Test' Dataset 
X_train, X_test_sim, y_train, y_test_sim = train_test_split(X, y, test_size=0.2, random_state=42)

# 3. Concatenate X and y elements of 'Simulated Test' and 'Train' Datasets
df_train    = pd.concat([y_train, X_train], axis=1)
df_test_sim = pd.concat([y_test_sim, X_test_sim], axis=1)



---

#### <b> Part 2: Define Functions: Data Cleaning  </b>

Data Cleaning Functions Required

* <b>Fomatting</b>: Convert column names to lowercase
* <b>Floor Area</b>: Exclude outliers by “Flat Type-Model”
* <b>Accessibility to Amenities</b>: Perform row-wise deletion of observations with missing values related to "distance to nearest mall", since these form an insignificant proportion of all observations (~0.6%)

</br>


---

<b>(a) Define Function: Convert Column Names to Lowercase</b>

In [4]:
# Define Function: Convert Column Names to Lowercase
def lowercase_column_names(dataframe):
    dataframe.columns = dataframe.columns.str.lower()


<b>(b) Define Function: Exclude Outliers by “Flat Type-Model”</b>

In [5]:
# 1. Build Static Dictionary of Reasonable Bounds by 'Flat Type-Model' based on "X_train"

# Reference: Notebook "1_EDA" -> Hypothesis (1) : Apartment's Floor Area is Positively Related with Resale Price -> (c) Data Cleaning : Floor Area (Square Meters)
# Reasonable Bound (By Flat Type-Model): "resonable bound" = [q1 - (1 x IQR) , q3 - (1 x IQR) ]

# Add Column: Flat Type-Model 
X_train["flat_type_model"] = X_train["flat_type"] + " " + X_train["flat_model"]
flat_type_model_list = [type for type in X_train["flat_type_model"].unique()]
flat_type_model_list.sort()

# Add Dictionary: Reasonable Bounds by 'Flat Type-Model'
floor_area_bounds = {flat_type_model:[ ] for flat_type_model in flat_type_model_list}

for flat_type_model in flat_type_model_list:
    q1 = X_train.loc[(X_train["flat_type_model"]==flat_type_model), ["floor_area_sqm"]].quantile(0.25)[0]
    q3 = X_train.loc[(X_train["flat_type_model"]==flat_type_model), ["floor_area_sqm"]].quantile(0.75)[0]   
    iqr = q3 - q1

    threshold = 1.0
    lower_bound = round(q1 - (threshold)*(iqr), 0)
    upper_bound = round(q3 + (threshold)*(iqr), 0)

    floor_area_bounds[flat_type_model].append(lower_bound)
    floor_area_bounds[flat_type_model].append(upper_bound)
    

In [6]:
# 2. Define Function: Exclude Floor Area Outliers

def floor_area_excl_outliers(dataframe):

    # Add Column: Flat Type-Model 
    dataframe["flat_type_model"] = dataframe["flat_type"] + " " + dataframe["flat_model"]

    # Exclude Floor Area Outliers
    for flat_type_model,bounds in floor_area_bounds.items():
        lower_bound = bounds[0]
        upper_bound = bounds[1]
        
        dataframe.drop(dataframe[(dataframe["flat_type_model"]==flat_type_model) & (dataframe["floor_area_sqm"]>upper_bound)].index, inplace=True)
        dataframe.drop(dataframe[(dataframe["flat_type_model"]==flat_type_model) & (dataframe["floor_area_sqm"]<lower_bound)].index, inplace=True)
    

<b>(c) Define Function: Perform Row-Wise Deletion of Observations with Missing Values Related to "Distance to Nearest Mall"</b>

In [7]:
# Define Function: Perform Row-Wise Deletion of Observations with Missing Values Related to "Distance to Nearest Mall"

# Reference: Notebook "1_EDA" -> Hypothesis (7) : Accessibility to Amenities is Positively Related to Resale Price -> (c) Data Cleaning
# Treatment of Missing Values: Row-wise deletion of observations 

def mall_nearest_distance_excl_missing_obs(dataframe):
    dataframe.drop(dataframe[(dataframe["mall_nearest_distance"]!=dataframe["mall_nearest_distance"])].index, inplace=True)


<b>(d) Define Function: Cleaning Procedures (Consolidated)</b>

In [8]:
# Define Function: Cleaning Procedures (Consolidated)

def clean_data(dataframe):

    dataframe = dataframe.copy()

    # Cleaning Procedures
    lowercase_column_names(dataframe)
    floor_area_excl_outliers(dataframe)
    mall_nearest_distance_excl_missing_obs(dataframe)

    return dataframe


---

#### <b> Part 3: Define Functions: Feature Engineering  </b>



<b>(a) Simple Features</b>

* <b>Floor Area</b>: (Floor Area)^2
* <b>Floor Level</b>: (Floor Level)^2
* <b>Exceptional Flat Types</b>: "DBSS", "Terrace", "Mansionette/Loft", "Duxton S-Type" Dummies
* <b>Remaining Lease Period</b>: (Remaining Lease Period)^2

* <b>Apartment's Neighbour Composition</b>
    * (a) If Apartment Block has Rental Units
    * (b) Apartment Block’s ‘5-Room & Larger’ Flat Type Proportion - Categorical: (i) [0-0.25] %. (ii) (0.25-0.50] %, (iii) (0.50-0.75] %, (iv) (0.75-1] %

* <b>Accessibility to Transport Infrastructure</b>: 
    * (a) Nearest MRT Station: (i) within 500m (Walking Distance), (ii) between 500 to 1000m , (iii) beyond 1000m
    * (b) Nearest MRT Station: (i) train + bus interchange, (ii) train-only interchange, (iii) bus-only interchange, (iv) not a train/bus interchange 
    * (c) Nearest Bus Stop within Close Proximity (150m)

* <b>Accessibility to Amenities</b>: 
    * (a) Nearest Mall: (i) within 500m (Walking Distance), (ii) between 500 to 1000m , (iii) beyond 1000m
    * (b) Nearest Hawker Centre: (i) within 500m (Walking Distance), (ii) between 500 to 1000m , (iii) beyond 1000m
    * (c) Nearest Hawker Centre's Size (No. of Food & Market Stalls): (i) small (<=100 stalls), (ii) large (>100 stalls)

* <b>Unobserved Area-based Features</b>: “Town” Dummies
* <b>Time-based Events</b>: “Transaction Year” Dummies

<b>(b) Complex Features</b>

* <b>Accessibility to the CBD</b>: Travel Cost to the CBD via the MRT Network

* <b>Accessibility to ‘Branded’ Schools</b>: 
    * (a) Number of ‘Branded’ primary schools: (i) within 1 km, (ii) between 1km to 2km
    * (b) Number of ‘Branded’ secondary schools: (i) within 1 km, (ii) between 1km to 2km

<br>


---

<b> (a) Define Functions: Engineer Simple Features </b>

Note: Constructing One-Hot Encoded Features

* The construction of two simple features ("Town", "Transaction Year" dummies) requires representing categories within a categorical data column as a set of dummies
* However, the list of categories within the "train", "simulated test", and "test" datasets may be different
* To [enforce feature space consistency](https://albertum.medium.com/preprocessing-onehotencoder-vs-pandas-get-dummies-3de1f3d77dcc) required for a model trained on "train" data to produce predictions on the "simulated test" and "test" datasets – Sklearn’s "OneHotEncoder" will be used to save category labels from the "train" dataset (via "fit_transform"), before applying these to the "simulated test" and "test" datasets (via "transform")


In [9]:
# Define Function: Engineer Simple Features

def engineer_features_simple(dataframe):

    dataframe = dataframe.copy()

    # (a) Floor Area
    dataframe["floor_area_sqm_2"] = dataframe["floor_area_sqm"]**2

    # (b) Floor Level
    dataframe["floor_level_mid"]   = dataframe["mid"]
    dataframe["floor_level_mid_2"] = dataframe["mid"]**2

    # (c) Exceptional Flat Types
    dataframe["flat_type_dbss"]            = (dataframe["flat_model"].str.contains("DBSS")).astype(int)
    dataframe["flat_type_terrace"]         = (dataframe["flat_model"].str.contains("Terrace")).astype(int)
    dataframe["flat_type_maisonette_loft"] = (dataframe["flat_model"].str.contains("Maisonette") | dataframe["flat_model"].str.contains("Loft")).astype(int)
    dataframe["flat_type_duxton_s1_s2"]    = (dataframe["flat_model"].str.contains("Type S1") | dataframe["flat_model"].str.contains("Type S2")).astype(int)
    
    # (d) Remaining Lease Period
    dataframe["remaining_lease_years"]   = 99 - (dataframe["tranc_year"] - dataframe["lease_commence_date"]) 
    dataframe["remaining_lease_years_2"] = dataframe["remaining_lease_years"]**2


    # (e) Apartment's Neighbour Composition

    ## 1. If Apartment Block has Rental Units
    dataframe["blk_rental_present"] = ((dataframe["1room_rental"] + dataframe["2room_rental"] + dataframe["3room_rental"] + dataframe["other_room_rental"])>0).astype(int)

    ## 2. Apartment Block’s '5-Room & Larger' Flat Type Proportion
    dataframe["blk_5rm_abv_p"]            = (dataframe["5room_sold"] + dataframe["exec_sold"] + dataframe["multigen_sold"])/dataframe["total_dwelling_units"]
    dataframe["blk_5rm_abv_p_frm000_025"] = ((dataframe["blk_5rm_abv_p"]>=0.00) & (dataframe["blk_5rm_abv_p"]<=0.25)).astype(int)
    dataframe["blk_5rm_abv_p_abv025_050"] = ((dataframe["blk_5rm_abv_p"]>0.25)  & (dataframe["blk_5rm_abv_p"]<=0.50)).astype(int)
    dataframe["blk_5rm_abv_p_abv050_075"] = ((dataframe["blk_5rm_abv_p"]>0.50)  & (dataframe["blk_5rm_abv_p"]<=0.75)).astype(int)
    dataframe["blk_5rm_abv_p_abv075_100"] = ((dataframe["blk_5rm_abv_p"]>0.75)  & (dataframe["blk_5rm_abv_p"]<=1.00)).astype(int)


    # (f) Accessibility to Transport Infrastructure

    ## 1. Nearest MRT Station: Distance
    dataframe["mrt_nearest_frm000_500"]  = ((dataframe["mrt_nearest_distance"]>=0) & (dataframe["mrt_nearest_distance"]<=500)).astype(int)
    dataframe["mrt_nearest_abv500_1000"] = ((dataframe["mrt_nearest_distance"]>500) & (dataframe["mrt_nearest_distance"]<=1000)).astype(int)
    dataframe["mrt_nearest_abv1000"]     = (dataframe["mrt_nearest_distance"]>1000).astype(int)

    ## 2. Nearest MRT Station: Interchange Status
    dataframe["mrt_nearest_interchange_train_bus"]  = ((dataframe["mrt_interchange"]==1) & (dataframe["bus_interchange"]==1)).astype(int)
    dataframe["mrt_nearest_interchange_train_only"] = ((dataframe["mrt_interchange"]==1) & (dataframe["bus_interchange"]==0)).astype(int)
    dataframe["mrt_nearest_interchange_bus_only"]   = ((dataframe["mrt_interchange"]==0) & (dataframe["bus_interchange"]==1)).astype(int)
    dataframe["mrt_nearest_interchange_nil"]        = ((dataframe["mrt_interchange"]==0) & (dataframe["bus_interchange"]==0)).astype(int)

    ## 3. Nearest Bus Stop
    dataframe["bus_stop_nearest_frm000_150"] = (dataframe["bus_stop_nearest_distance"]<=150).astype(int)


    # (g) Accessibility to Amenities

    ## 1. Nearest Mall
    dataframe["mall_nearest_frm000_500"]  = ((dataframe["mall_nearest_distance"]>=0) & (dataframe["mall_nearest_distance"]<=500)).astype(int)
    dataframe["mall_nearest_abv500_1000"] = ((dataframe["mall_nearest_distance"]>500) & (dataframe["mall_nearest_distance"]<=1000)).astype(int)
    dataframe["mall_nearest_abv1000"]     = (dataframe["mall_nearest_distance"]>1000).astype(int)

    ## 2. Nearest Hawker Centre
    dataframe["hawker_nearest_frm000_500"]  = ((dataframe["hawker_nearest_distance"]>=0) & (dataframe["hawker_nearest_distance"]<=500)).astype(int)
    dataframe["hawker_nearest_abv500_1000"] = ((dataframe["hawker_nearest_distance"]>500) & (dataframe["hawker_nearest_distance"]<=1000)).astype(int)
    dataframe["hawker_nearest_abv1000"]     = (dataframe["hawker_nearest_distance"]>1000).astype(int)

    # 3. Nearest Hawker Centre's Size
    dataframe["hawker_nearest_size_large"]  = ((dataframe["hawker_food_stalls"] + dataframe["hawker_market_stalls"])>100).astype(int)


    # (h) One-Hot Encoded Features: Unobserved Area-based Features & Time-based Events

    # 1. Setup: One-Hot Encoded Features' Category Labels

    # Instantiate Sklearn's OneHotEncoder
    ohe = OneHotEncoder(handle_unknown="ignore", sparse_output=False)

    # Save Category Labels from "Train" dataset (for Subsequent Consistent Application to the "Simulated Test" and "Test" Datasets)
    data_train = X_train.copy()
    data_train.columns = data_train.columns.str.lower()

    cat_features = ["town", "tranc_year"]
    ohe_fit_transform_data_train = ohe.fit(data_train[cat_features])

    # 2. Add: One-Hot Encoded Features
    dataframe = dataframe.reset_index(drop=True)  # index resetting necessary for subsequent horizontal concatenation

    ohe_data = ohe.transform(dataframe[cat_features])
    ohe_df   = pd.DataFrame(ohe_data, columns=ohe.get_feature_names_out(cat_features))
    ohe_df.columns = ohe_df.columns.str.lower()

    dataframe = pd.concat([dataframe, ohe_df], axis=1)

    return dataframe



---

<b> (b-1) Define Functions: Engineer Complex Features -- Travel Cost to CBD via the MRT Network</b>

<br><b>Background</b>

The MRT Network Considered
* Network as at 2024
* HDB resale data spans from 2012 to 2021, and the MRT network has evolved during the period (with [addition of the 'Downtown' line in 2017](https://www.lta.gov.sg/content/ltagov/en/getting_around/public_transport/rail_network/downtown_line.html), and [opening of the 'Thompson-East Coast' line in 2020](https://www.lta.gov.sg/content/ltagov/en/upcoming_projects/rail_expansion/thomson_east_coast_line.html))
* We nonetheless considered the network as at 2024, as:
    * Data checks reveal that "name of nearest MRT station" in the source data is based on the MRT network as at time of the data's preparation (2022), instead of resale transaction year
    * We expect home buyers to consider evolution of the MRT network when making a purchase (alongside other future-oriented area development plans)

<br> <b>Computation of Travel Cost </b>

* Conceptual Definition: Travel Cost to the CBD via the MRT Network
    * Travel Cost = Minimum(Number of MRT stations away from a CBD station + Transfer Penalty for Switching MRT Lines)
    * 'Transfer Penalty' - Penalty for making a switch modelled as equivalent to travelling to an additional MRT station

* Computation Approach
    * Utilises [Dijkstra's algorithm](https://en.wikipedia.org/wiki/Dijkstra%27s_algorithm) to construct: `mrt_travel_cost_cbd`
    * Implementation of the algorithm is credited to Eryk Kopczyński [(code reference)](https://www.python.org/doc/essays/graphs/), and uses a [deque](https://docs.python.org/3/library/collections.html#collections.deque) from Python's collections module for efficient breadth-first search
    * Credit for writing this section of code goes to [Wei Chiong Tan](t-wei-ch.github.io/my-portfolio/)

* Definition: CBD Stations 

    |Line|Station(s)|
    |---|---|
    |NSL|Raffles Place, City Hall, Marina Bay|
    |EWL|Outram Park, Tanjong Pagar, Raffles Place, City Hall|
    |CCL|Telok Blangah, Harbourfront, Bayfront, Promenade, Esplanade, Marina Bay|
    |DTL|Telok Ayer, Downtown, Bayfront, Promenade|
    |TEL|Maxwell, Shenton Way, Marina Bay|

<br> <b>Code Implementation</b>

* Code for computing 'Travel Cost to CBD via the MRT Network' has been written in a separate script, under: '1_Data/mrt_tools.py'
* Codes in this notebook import the script's computation function


<b>(1) Imports

In [10]:
import collections
from Data_Processing_MRT_Tools import mrt_scores_per_station

<b>(2) Define Function: Travel Cost to CBD via the MRT Network

In [11]:
def insert_mrt_travel_cost_cbd(dataframe):

    dataframe = dataframe.copy()

    # Insert `mrt_travel_cost_cbd` column into `dataframe`
    dataframe["mrt_travel_cost_cbd"]   = dataframe.apply(lambda row: mrt_scores_per_station[row["mrt_name"]], axis=1)
    dataframe["mrt_travel_cost_cbd_2"] = dataframe["mrt_travel_cost_cbd"]**2

    return dataframe
    


---

<b> (b-2) Define Functions: Engineer Complex Features -- Accessibility to 'Branded' Schools</b>

<br><b>Background: Definition of 'Branded' Schools</b>

* We rely on a classification approach used in a [research study](https://www.childrensociety.org.sg/wp-content/uploads/2022/07/Schools-and-Class-Divide_Research-Monograph-11_FINAL_24-Aug-2016v3.pdf) by the Singapore Children's Society to identify "branded/elite" schools:
    * Primary Schools: offer the Gifted Education Programme (GEP), are affiliated to Integrated Programme secondary schools, or are government-aided
    * Secondary Schools: offer the Integrated Programme, or are autonomous schools

<br><b>Code Implementation</b>

* Step 1: obtain full list of schools and their geographic co-ordinates
* Step 2: specify list of 'branded' schools
* Step 3: compute distances of 'branded' schools from resale flats sold, to build a picture of the no. of 'branded' schools within a given geographic radius
* Credit for writing this section of code goes to [Wei Chiong Tan](t-wei-ch.github.io/my-portfolio/)


<b>(1) Obtain Full List of Schools and their Geographic Coordinates</b>

In [12]:
# 1. Build Dictionary of Schools-Geo Coordinates from Source Data

# Isolate primary school names and their relevant coordinates to a dictionary
pri_sch_dict = df_train_init.loc[:,['pri_sch_name', 'pri_sch_latitude', 'pri_sch_longitude']].to_dict(orient='list')

# Isolate secondary school names and their relevant coordinates to a dictionary
sec_sch_dict = df_train_init.loc[:,['sec_sch_name', 'sec_sch_latitude', 'sec_sch_longitude']].to_dict(orient='list')

# Match schools with their coordinates
lst_of_pri_sch_coords = zip(pri_sch_dict['pri_sch_latitude'], pri_sch_dict['pri_sch_longitude'])
lst_of_sec_sch_coords = zip(sec_sch_dict['sec_sch_latitude'], sec_sch_dict['sec_sch_longitude'])

pri_sch_coords = {name: coords for name, coords in zip(pri_sch_dict['pri_sch_name'], lst_of_pri_sch_coords)}
sec_sch_coords = {name: coords for name, coords in zip(sec_sch_dict['sec_sch_name'], lst_of_sec_sch_coords)}

# 2. Add Coordinates of Relevant Missing "Branded" Schools
pri_sch_coords["Anglo-Chinese School"] = (1.31875235, 103.835076922932)
pri_sch_coords["Raffles Girls' Primary School"] = (1.33004178, 103.806397828938)
pri_sch_coords["Singapore Chinese Girls' School"] = (1.3210897, 103.827867312987)
pri_sch_coords["Methodist Girls' School"] = (1.3329500, 103.7825694)

sec_sch_coords["Nanyang Girls' High School"] = (1.3305, 103.8024)
sec_sch_coords["Hwa Chong Institution"] = (1.32654, 103.803491)
sec_sch_coords["Saint Joseph's Institution"] = (1.3236, 103.8273)
sec_sch_coords["National Junior College"] = (1.330278, 103.804167)
sec_sch_coords["NUS High School"] = (1.306911, 103.769356)
sec_sch_coords["Singapore Chinese Girls' School"] = (1.3210897, 103.827867312987)
sec_sch_coords["Victoria School"] = (1.308575, 103.927467)
sec_sch_coords["Maris Stella High School"] = (1.3421839569999998, 103.8780964)
sec_sch_coords["Tanjong Katong Girls' School"] = (1.30785, 103.89619)

In [13]:
# Test: print coordinates of first 5 primary schools in dictionary
collections.OrderedDict(list(pri_sch_coords.items())[0: 5])

OrderedDict([('Geylang Methodist School', (1.317658971, 103.8825037)),
             ('Kuo Chuan Presbyterian Primary School',
              (1.349783074, 103.8545292)),
             ('Keming Primary School', (1.3452450530000002, 103.7562645)),
             ('Catholic High School', (1.3547888769999998, 103.8449341)),
             ('Naval Base Primary School', (1.4162801530000002, 103.8387977))])

In [14]:
# Test: print coordinates of first 5 secondary schools in dictionary
collections.OrderedDict(list(sec_sch_coords.items())[0: 5])

OrderedDict([('Geylang Methodist School', (1.317658971, 103.8825037)),
             ('Kuo Chuan Presbyterian Secondary School',
              (1.350109648, 103.8548917)),
             ('Yusof Ishak Secondary School',
              (1.3423337780000002, 103.7600131)),
             ('Catholic High School', (1.3547888769999998, 103.8449341)),
             ('Orchid Park Secondary School', (1.414888187, 103.8383349))])

<b>(2) Specify List of 'Branded' Schools</b>

In [15]:
branded_pri_schs = ['Anglo-Chinese School',
                    'Catholic High School',
                    "CHIJ Saint Nicholas Girls' School",
                    'Henry Park Primary School',
                    'Nanyang Primary School',
                    "Raffles Girls' Primary School",
                    'Rosyth School',
                    "Singapore Chinese Girls' School",
                    "Saint Hilda's Primary School",
                    "Saint Joseph's Institution Junior",
                    "Saint Stephen's School",
                    'Tao Nan School',
                    'Ai Tong School',
                    'Canossa Catholic Primary School',
                    'CHIJ',
                    'CHIJ Our Lady of Good Counsel',
                    'CHIJ Our Lady of The Nativity',
                    'CHIJ Our Lady Queen of Peace',
                    'Chongfu School',
                    'De La Salle School',
                    'Fairfield Methodist School',
                    'Geylang Methodist School',
                    "Holy Innocents' Primary School",
                    'Hong Wen School',
                    'Kheng Cheng School',
                    'Kong Hwa School',
                    'Kuo Chuan Presbyterian Primary School',
                    'Maha Bodhi School',
                    'Maris Stella High School',
                    'Marymount Convent School',
                    'Mee Toh School',
                    'Nan Chiau Primary School',
                    'Ngee Ann Primary School',
                    "Paya Lebar Methodist Girls' School",
                    'Pei Chun Public School',
                    'Pei Hwa Presbyterian Primary School',
                    'Poi Ching School',
                    'Red Swastika School',
                    "Saint Andrew's Junior School",
                    "Saint Anthony's Canossian Primary School",
                    "Saint Gabriel's Primary School",
                    "Saint Margaret's Primary School"]

branded_sec_schs = ['Anglo-Chinese School',
                    'Catholic High School',
                    "Cedar Girls' Secondary School",
                    "CHIJ Saint Nicholas Girls' School",
                    'Dunman High School',
                    'Hwa Chong Institution',
                    "Methodist Girls' School",
                    "Nanyang Girls' High School",
                    'National Junior College',
                    'NUS High School',
                    'Raffles Institution',
                    "Raffles Girls' School",
                    'River Valley High School',
                    "Singapore Chinese Girls' School",
                    "Saint Joseph's Institution",
                    'Temasek Junior College',
                    'Victoria School',
                    'Anderson Secondary School',
                    'Anglican High School',
                    'Bukit Panjang Government High School',
                    'CHIJ Katong Convent',
                    'CHIJ Secondary',
                    'Chung Cheng High School',
                    'Commonwealth Secondary School',
                    "Crescent Girls' School",
                    'Dunman Secondary School',
                    'Fairfield Methodist School',
                    'Maris Stella High School',
                    'Nan Hua High School',
                    'Ngee Ann Secondary School',
                    "Paya Lebar Methodist Girls' School",
                    "Saint Anthony's Canossian Secondary School",
                    "Saint Hilda's Secondary School",
                    "Saint Margaret's Secondary School",
                    "Tanjong Katong Girls' School",
                    'Tanjong Katong Secondary School',
                    'Temasek Secondary School',
                    'Xinmin Secondary School',
                    'Yishun Town Secondary School',
                    'Zhonghua Secondary School']

<b>(3) Compute Distance of 'Branded' schools from Resale Flats Sold, Build Picture of No. of 'Branded' Schools within a given Geographic Radius</b>

(a) Define Function: Calculate Distance Between 2 Latitude-Longitude Coordinates

<details><summary>Mathematical background of latitude and longitude calculations:</summary>

<br><u> Theoretical Challenges </u>

As is common knowledge since the circumnavigation of the Earth by the Magellan Expedition in 1519-1522, the Earth is not flat. However, modeling the Earth as a perfect sphere isn't exactly correct either, as the actual [shape of the Earth](https://en.wikipedia.org/wiki/Figure_of_the_Earth) is an oblate spheroid, and an imperfect one at that. This makes [measuring](https://en.wikipedia.org/wiki/Geographical_distance) exact distances on the surface of the Earth slightly challenging, and is one of the major problems that the creators of the Global Positioning System (GPS) had to solve.

Even more troubling, while relatively small distances on the surface of the Earth would be reasonably approximated to a small margin of error by using the Pythagorean theorem as the Earth is locally approximated by a flat plane, said theorem only works if the points given are Cartesian coordinates $p_1=(x_1, y_1)$ and $p_2=(x_2, y_2)$. The Pythagorean distance would then be given by $$d(p_1,p_2)=\sqrt{(x_1-x_2)^2+(y_1-y_2)^2}$$ That is not the case here, as we are given latitude and longitude, which assume that we are working in spherical coordinates. 

<br><u> Approach Undertaken</u>

Specifically, the point with latitude $\phi$ and longitude $\lambda$ would correspond to the Cartesian coordinates on the sphere with radius $r$ in 3-dimensional Euclidean space. $$(r\cos(\phi)\cos(\lambda),\ r\cos(\phi)\sin(\lambda),\ r\sin(\phi))$$  On the sphere, the shortest distance between two points would be computed instead by the [great circle distance](https://en.wikipedia.org/wiki/Great-circle_distance), the distance along the great circle on the sphere that connects the two points.

A relatively efficient computation for the great circle distance between two points specified by latitude and longitude is achieved by the [Haversine formula](https://en.wikipedia.org/wiki/Haversine_formula), which works just fine when two points are close to each other on the sphere, as is per our case for Singapore. Assuming that the latitudes and longitudes are all given in radians, the Haversine formula for the distance $d$ between two points on the sphere with radius $r$ given by latitudes and longitudes $p_1=(\phi_1, \lambda_1)$ and $p_2=(\phi_2, \lambda_2)$ is given as follows: $$d(p_1, p_2)=r\cdot\left(2\tan^{-1}\left(\frac{\sqrt{a}}{\sqrt{1-a}}\right)\right)$$ where $$a=\frac{1-\cos(\phi_2-\phi_1)}{2}+\cos(\phi_1)\cos(\phi_2)\left(\frac{1-\cos(\lambda_2-\lambda_1)}{2}\right)$$ is called the [haversine](https://en.wikipedia.org/wiki/Versine) (or half a versine). 

This may seem a tad complicated, but it essentially just uses the arc-length formula $d=r\theta$, and that the computation of $\theta$ is only more involved due to the inputs being latitude and longitude.
</details>

In [16]:
# Define Function: Calculate Distance Between 2 Latitude-Longitude Coordinates using the Haversine formula

def haversine_dist(lat_1, long_1, lat_2, long_2, r=6371):

    '''Computes the great circle distance between two points `(lat_1, long_1)` and `(lat_2, long_2)` 
    on a spherical Earth, specified by latitudes `lat_1`, `lat_2` and longitudes `long_1`, `long_2`.

    Parameters
    ----------
    `lat_1`: float
        Latitude of the first point in degrees
    `long_1`: float
        Longitude of the first point in degrees
    `lat_2`: float
        Latitude of the second point in degrees
    `long_2`: float
        Longitude of the second point in degrees
    `r`: float
        Radius of the Earth. Default `6371` km.

    Returns
    -------
    `dist`: float
        Distance between the two points in km.
    '''

    # convert all latitudes and longitudes to radians
    lat_1, long_1, lat_2, long_2 = np.radians(lat_1), np.radians(long_1), np.radians(lat_2), np.radians(long_2)

    # calculate latitude and longitude differences
    dlat  = lat_2 - lat_1
    dlong = long_2 - long_1

    # calculate the haversine
    haversine = 0.5 * (1 - np.cos(dlat) + np.cos(lat_1)*np.cos(lat_2)*(1 - np.cos(dlong)))

    # calculate great circle distance
    dist = 2 * r * np.arctan2(np.sqrt(haversine), np.sqrt(1 - haversine))

    return dist
    

(b) Define Function: Identify Distance to 'Branded' Schools, Count No. of 'Branded' Schools within a Given Distance from a Coordinate

In [17]:
# Define Function: Identify Distance to 'Branded' Schools, Count No. of 'Branded' Schools within a Given Distance from a Coordinate

import functools
@functools.cache

def nearest_landmarks(lat, long, landmark='pri', radius=1.0, output='landmarks'):
    
    '''
    Parameters
    ----------
    `lat`: float
        Latitude of the selected point
    
    `long`: float
        Longitude of the selected point
    
    `landmark`: string. Default `'pri'`
        `'pri'`:       Primary schools
        `'brand_pri'`: Branded primary schools
        `'sec'`:       Secondary schools
        `'brand_sec'`: Branded secondary schools
    
    `radius`: float
        Search radius in km. Default `1.0` km
    
    `output`: string. Default `'landmarks'`
        `'landmarks'`: Outputs a list of `landmark`s that are within a `dist` km radius around `(lat, long)`
        `'count'`:     Outputs how many `landmark`s there are within a `dist` km radius around `(lat, long)`
        `'data'`:      Outputs a dictionary of `dist: landmark` pairs 
        `'shortest'`:  Outputs the shortest distance in metres to the nearest `landmark`, sets `radius` parameter to 100
    '''
    
    # radius switch
    if output == 'shortest':
        radius = 100.0

    # initialise empty output dictionary
    dist_landmarks = {}

    # initialise list of landmarks
    lst_landmarks = []

    # for conveniently switching between the different landmark types without using if-else statements
    
    def filter_dict(dic, lst_of_keys):
        '''Outputs a filtered subset of the dictionary `dic` with only keys in `lst_of_keys`
        '''
        filtered_dic = {key: val for key, val in dic.items() if key in lst_of_keys}

        return filtered_dic    
    
    landmark_switch = {'pri': pri_sch_coords,
                       'brand_pri': filter_dict(pri_sch_coords, branded_pri_schs),
                       'sec': sec_sch_coords,
                       'brand_sec': filter_dict(sec_sch_coords, branded_sec_schs)}
    
    for building in landmark_switch[landmark]:
        # retrieve coordinates from the respective dictionary
        landmark_lat, landmark_long = landmark_switch[landmark][building]

        # compute distance to 5 decimal places
        dist = np.round(1000*haversine_dist(lat, long, landmark_lat, landmark_long), 5)

        if dist <= 1000*radius:
            # append the respective building to the output list
            lst_landmarks.append(building)

            if dist not in dist_landmarks:
                # add school name if there's no school with that distance away
                dist_landmarks[dist] = [building]
            else:
                # else append to the current list value
                dist_landmarks[dist].append(building)
    
    # sort dist_landmarks by distance
    dist_landmarks = dict(sorted(dist_landmarks.items()))

    if output == 'landmarks':
        return lst_landmarks

    if output == 'count':
        return len(lst_landmarks)
    
    if output == 'data':
        return dist_landmarks
    
    if output == 'shortest':
        # distances are already sorted in increasing order
        # the if-else condition accounts for the (unlikely) case where there really are no landmarks within a 100km radius
        return list(dist_landmarks.keys())[0] if dist_landmarks else np.nan

    return None

(c) Define Function: Insert into Dataframe -- Distance to Nearest 'Branded' School, Count of No. of 'Branded' Schools within a Given Distance from Resale Unit Sold

In [18]:
def insert_branded_sch_cols(dataframe):

    dataframe = dataframe.copy()

    '''
    Insert branded school columns
    ----------
    '''
    
    # school-based variables
    for level in ['pri', 'sec']:
        dataframe[f'dist_to_nearest_brand_{level}_sch'] = dataframe.apply(lambda row: nearest_landmarks(row['latitude'],
                                                                                                        row['longitude'],
                                                                                                        landmark=f'brand_{level}',
                                                                                                        output='shortest'),
                                                                                                        axis=1)
        for dist in [1, 2]:
            dataframe[f'n_brand_{level}_sch_within_{dist-1}-{dist}km'] = dataframe.apply(lambda row: nearest_landmarks(row['latitude'], 
                                                                                                                       row['longitude'], 
                                                                                                                       landmark=f'brand_{level}',
                                                                                                                       radius=dist,
                                                                                                                       output='count') - 
                                                                                                     nearest_landmarks(row['latitude'], 
                                                                                                                       row['longitude'], 
                                                                                                                       landmark=f'brand_{level}',
                                                                                                                       radius=dist-1,
                                                                                                                       output='count'),
                                                                                                                       axis=1)

    return dataframe
                                                                                                                       


---

#### <b> Part 4: Define Functions: Others  </b>

<b>Overview: Other Functions</b>
* (a) Retain Selected Features
* (b) Scale Numeric Features

<b>(a) Define Function: Retain Selected Features</b>

In [19]:
def retain_selected_features(dataframe):

    dataframe = dataframe.copy()
    
    dataframe.rename(columns={"mrt_interchange":"mrt_nearest_interchange_mrt", "bus_interchange":"mrt_nearest_interchange_bus"}, inplace="True")

    id_no = ["id"]

    selected_y_feature  = ["resale_price"]

    selected_x_features = ["floor_area_sqm",
                           "floor_area_sqm_2",
                           "floor_level_mid",
                           "floor_level_mid_2",
                           "flat_type_dbss",
                           "flat_type_terrace",
                           "flat_type_maisonette_loft",
                           "flat_type_duxton_s1_s2",
                           "remaining_lease_years",
                           "remaining_lease_years_2",
                           "blk_rental_present",
                           "mrt_nearest_frm000_500",
                           "mrt_nearest_abv500_1000",
                           "mrt_nearest_interchange_train_bus",     # reference cat: "mrt_nearest_interchange_nil"
                           "mrt_nearest_interchange_train_only",
                           "mrt_nearest_interchange_bus_only",
                           "bus_stop_nearest_frm000_150",
                           "mall_nearest_frm000_500",
                           "mall_nearest_abv500_1000",
                           "hawker_nearest_frm000_500",
                           "hawker_nearest_abv500_1000",
                           "hawker_nearest_size_large",
                           "mrt_travel_cost_cbd",
                           "mrt_travel_cost_cbd_2",
                           "n_brand_pri_sch_within_0-1km",
                           "n_brand_pri_sch_within_1-2km",
                           "n_brand_sec_sch_within_0-1km",
                           "n_brand_sec_sch_within_1-2km",
                           "town_bedok",                            # reference cat: "town_ang mo kio"
                           "town_bishan",
                           "town_bukit batok",
                           "town_bukit merah",
                           "town_bukit panjang",
                           "town_bukit timah",
                           "town_central area",
                           "town_choa chu kang",
                           "town_clementi",
                           "town_geylang",
                           "town_hougang",
                           "town_jurong east",
                           "town_jurong west",
                           "town_kallang/whampoa",
                           "town_marine parade",
                           "town_pasir ris",
                           "town_punggol",
                           "town_queenstown",
                           "town_sembawang",
                           "town_sengkang",
                           "town_serangoon",
                           "town_tampines",
                           "town_toa payoh",
                           "town_woodlands",
                           "town_yishun",
                           "tranc_year_2013",                       # reference cat: "tranc_year_2012"    
                           "tranc_year_2014",
                           "tranc_year_2015",
                           "tranc_year_2016",
                           "tranc_year_2017",
                           "tranc_year_2018",
                           "tranc_year_2019",
                           "tranc_year_2020",
                           "tranc_year_2021"
                           ]

    try: 
        dataframe = dataframe[id_no + selected_y_feature + selected_x_features]

    except:
        dataframe = dataframe[id_no + selected_x_features]  # to accomodate absence of 'selected_y_feature' in "df_test"
    
    return dataframe
        

<b>(b) Define Function: Scale Numeric Features</b>

Purpose & Process of Scaling Numeric Features
* Regularisation techniques (Ridge, LASSO) will be applied subsequently in an attempt to improve model generalisability 
* The application of such techniques requires numeric features to be scaled, in order for estimated coefficients on these terms to be equally influenced by the regularising penalty term (lambda)
* Procedurally: scaling factors will be firstly derived from "training" data, before being applied to the "test (simulated)" and "test" datasets

In [20]:
def scale_numeric_features(dataframe):

    # 1. Instantiate Standard Scaler
    ss = StandardScaler()

    # ----------------------------------------------------------------------
    # 2. Derive Scaling Factors from "Training" Data

    ## Implement Data Processing Procedures Prior to Scaling
    data_train = df_train.copy()

    ## a. Clean & Engineer Features (Simple)
    data_train = clean_data(data_train)
    data_train = engineer_features_simple(data_train)

    ## b. Engineer Features (Complex)
    data_train = insert_mrt_travel_cost_cbd(data_train)
    data_train = insert_branded_sch_cols(data_train)

    ## c. Retain Selected Features
    data_train = retain_selected_features(data_train)

    ## d. Identify 'Y', 'X(Categorical)' & 'X(Numeric)' Features
    y_feature = ["resale_price"]

    x_cat_features = ["flat_type_dbss",
                      "flat_type_terrace",
                      "flat_type_maisonette_loft",
                      "flat_type_duxton_s1_s2",
                      "blk_rental_present",
                      "mrt_nearest_frm000_500",
                      "mrt_nearest_abv500_1000",
                      "mrt_nearest_interchange_train_bus",
                      "mrt_nearest_interchange_train_only",
                      "mrt_nearest_interchange_bus_only",
                      "bus_stop_nearest_frm000_150",
                      "mall_nearest_frm000_500",
                      "mall_nearest_abv500_1000",
                      "hawker_nearest_frm000_500",
                      "hawker_nearest_abv500_1000",
                      "hawker_nearest_size_large"]

    for feature in data_train.columns:
        if "town_" in feature:
            x_cat_features.append(feature)
        if "tranc_year_" in feature:
            x_cat_features.append(feature)

    x_num_features = ["floor_area_sqm",
                      "floor_area_sqm_2",
                      "floor_level_mid",
                      "floor_level_mid_2",
                      "remaining_lease_years",
                      "remaining_lease_years_2",
                      "mrt_travel_cost_cbd",
                      "mrt_travel_cost_cbd_2",
                      "n_brand_pri_sch_within_0-1km",
                      "n_brand_pri_sch_within_1-2km",
                      "n_brand_sec_sch_within_0-1km",
                      "n_brand_sec_sch_within_1-2km"]
    
    # e. Derive and Save 'X(Numeric)' Features' Scaling Factors
    data_train_x_num_features = data_train[x_num_features]
    data_train_X_num_features_scaled_fit = ss.fit(data_train_x_num_features)

    # ----------------------------------------------------------------------
    # 3. Scale Dataframe's Numeric Columns

    dataframe = dataframe.copy()

    ## a. Split Dataframe into Separate 'Y', 'X(Categorical)' & 'X(Numeric)' Dataframes

    try: 
        dataframe_id_no          = dataframe["id"]
        dataframe_y_feature      = dataframe[y_feature]
        dataframe_x_cat_features = dataframe[x_cat_features]
        dataframe_x_num_features = dataframe[x_num_features]

    except: # to accomodate absence of 'Y' in "df_test"
        dataframe_id_no          = dataframe["id"]
        dataframe_x_cat_features = dataframe[x_cat_features]
        dataframe_x_num_features = dataframe[x_num_features]        

    # b. Scale 'X(Numeric)' Dataframe's Features
    dataframe_x_num_features_scaled_array = ss.transform(dataframe_x_num_features)
    dataframe_x_num_features_scaled_df = pd.DataFrame(dataframe_x_num_features_scaled_array, columns=ss.get_feature_names_out(x_num_features))

    # c. Concatenate 'Y', 'X(Categorical)' & 'X(Numeric)' Dataframes, Re-order Selected Features

    id_no      = ["id"]

    y_feature  = ["resale_price"]

    x_features = ["floor_area_sqm",
                  "floor_area_sqm_2",
                  "floor_level_mid",
                  "floor_level_mid_2",
                  "flat_type_dbss",
                  "flat_type_terrace",
                  "flat_type_maisonette_loft",
                  "flat_type_duxton_s1_s2",
                  "remaining_lease_years",
                  "remaining_lease_years_2",
                  "blk_rental_present",
                  "mrt_nearest_frm000_500",
                  "mrt_nearest_abv500_1000",
                  "mrt_nearest_interchange_train_bus",
                  "mrt_nearest_interchange_train_only",
                  "mrt_nearest_interchange_bus_only",
                  "bus_stop_nearest_frm000_150",
                  "mall_nearest_frm000_500",
                  "mall_nearest_abv500_1000",
                  "hawker_nearest_frm000_500",
                  "hawker_nearest_abv500_1000",
                  "hawker_nearest_size_large",
                  "mrt_travel_cost_cbd",
                  "mrt_travel_cost_cbd_2",
                  "n_brand_pri_sch_within_0-1km",
                  "n_brand_pri_sch_within_1-2km",
                  "n_brand_sec_sch_within_0-1km",
                  "n_brand_sec_sch_within_1-2km"]

    for feature in dataframe_x_cat_features.columns:
        if "town_" in feature:
            x_features.append(feature)
        if "tranc_year_" in feature:
            x_features.append(feature)

    try:
        dataframe = pd.concat([dataframe_id_no, dataframe_y_feature ,dataframe_x_num_features_scaled_df, dataframe_x_cat_features], axis=1)
        dataframe = dataframe[id_no + y_feature + x_features]

    except: # to accomodate absence of 'Y' in "df_test"
        dataframe = pd.concat([dataframe_id_no, dataframe_x_num_features_scaled_df, dataframe_x_cat_features], axis=1)
        dataframe = dataframe[id_no + x_features]
    
    return dataframe


---

#### <b> Part 5: Perform Data Processing </b>

<b>Overview</b>

* The 3 mutually exclusive raw datasets ("df_train", "df_test_sim", "df_test") will be subject to the same data processing procedures
* In addition, the processing of each raw dataset will produce two datasets as outputs:

    |Dataset Description|X(Numeric) Features|Dataset Suffix|Purpose                                                                                                     |
    |-------------------|-------------------|--------------|------------------------------------------------------------------------------------------------------------|
    |Processed          |Unscaled           |"_p"          |For identifying features that are significantly associated with resale price (practically and statistically)|
    |Processed          |Scaled             |"_ps"         |For attempting to improve model's generalisability: via implementation of regularisation procedures         |

<br>
<b>(a) Define Data Processing Function</b>

In [21]:
def process_data(dataframe, scale_num="no"):

    dataframe = dataframe.copy()

    # Clean & Engineer Features (Simple)
    dataframe = clean_data(dataframe)
    dataframe = engineer_features_simple(dataframe)

    # Engineer Features (Complex)
    dataframe = insert_mrt_travel_cost_cbd(dataframe)
    dataframe = insert_branded_sch_cols(dataframe)

    # Retain Selected Features 
    dataframe = retain_selected_features(dataframe)

    if scale_num=="no":
        return dataframe
    
    if scale_num=="yes": # Scale Numeric Features
        dataframe = scale_numeric_features(dataframe)
        return dataframe       
        

<b>(b) Process Data</b>

In [22]:
df_train_p  = process_data(df_train, scale_num="no")
df_train_ps = process_data(df_train, scale_num="yes")

df_test_sim_p  = process_data(df_test_sim, scale_num="no")
df_test_sim_ps = process_data(df_test_sim, scale_num="yes")

df_test_p  = process_data(df_test, scale_num="no")
df_test_ps = process_data(df_test, scale_num="yes")

<b>(c) Export Data as CSV Files</b>

In [23]:
# Define: Dictionary of Dataframe Names & Dataframes
data = {"df_train_p":df_train_p,
        "df_train_ps":df_train_ps,
        "df_test_sim_p":df_test_sim_p,
        "df_test_sim_ps":df_test_sim_ps,
        "df_test_p":df_test_p,
        "df_test_ps":df_test_ps
        }

# Export Data as CSV Files
for df_name, df in data.items():
    path = f"1_Data\{df_name}.csv"
    df.to_csv(path, index=False)


---